In [4]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
from urllib3.exceptions import NewConnectionError

INFLUX_URL = "http://influxdb2:8086"
INFLUX_TOKEN = "MyInitialAdminToken0=payosno"

print("--- Iniciando conexión a InfluxDB ---")

client = None
try:

    client = influxdb_client.InfluxDBClient(
        url=INFLUX_URL,
        token=INFLUX_TOKEN,
        org="docs"
    )
    health = client.health()
    
    if health.status == "pass":
        print("[INFO] ¡Conexión exitosa!")
        print(f"[INFO]  Versión del servidor: {health.version}")
    else:
        print(f"[ERROR] Conexión fallida. Estado: {health.status}")
        print(f"[INFO] Mensaje: {health.message}")

except (InfluxDBError, NewConnectionError) as e:
    print("[ERROR] Error al conectar con InfluxDB:")
    print(f"   Detalle: {e}")
    
# finally:
#     # 3. Es buena práctica cerrar siempre el cliente
#     if client:
#         client.close()
#         print("--- Conexión cerrada ---")

--- Iniciando conexión a InfluxDB ---
[INFO] ¡Conexión exitosa!
[INFO]  Versión del servidor: v2.7.12


# 1. Obtenemos las métricas de rendimiento de la máquina

In [5]:
import psutil
import time

# Obtener estadísticas de uso de la máquina
def obtener_metricas(host_id):
    # Uso de cpu (promedio)
    cpu_usage = psutil.cpu_percent(interval=1)

    # Uso de RAM
    mem = psutil.virtual_memory()
    ram_used_gb = round(mem.used / (1024**3), 2)
    ram_percent = mem.percent

    # Uso de disco (en el punto de montaje)
    disk = psutil.disk_usage('/')
    disk_percent = disk.percent

    return {
        'host': host_id,
        'cpu_percent': cpu_usage,
        'ram_used_gb': ram_used_gb,
        'ram_percent': ram_percent,
        'disk_percent': disk_percent
    }

# 2. Modelo y escitura en lote

In [6]:
from influxdb_client import Point
import time


try:
    print("[INFO] Press Ctrl+C to interrupt.")
    while(True):
        time.sleep(1)

        metrics = obtener_metricas("servidor-01") 

        p = Point("rednimiento_servidor") \
            .tag("host_id", metrics["host"]) \
            .tag("entorno", "producción") \
            .field("cpu_percent", metrics["cpu_percent"]) \
            .field("ram_percent", metrics["ram_percent"]) \
            .field("disck_percent", metrics["disk_percent"])
        
        write_api = client.write_api(write_api=SYNCHRONOUS)

        write_api.write(bucket="iot_data", org="docs", record=p)

        print(">> Escribiendo...")
except KeyboardInterrupt:
    write_api.close()
    print("Se interrumpió por el usuario")




[INFO] Press Ctrl+C to interrupt.
>> Escribiendo...
>> Escribiendo...
>> Escribiendo...
>> Escribiendo...
>> Escribiendo...
>> Escribiendo...
>> Escribiendo...
>> Escribiendo...
>> Escribiendo...
>> Escribiendo...
>> Escribiendo...
>> Escribiendo...
>> Escribiendo...
>> Escribiendo...
>> Escribiendo...
>> Escribiendo...
Se interrumpió por el usuario


# Lectura de una base InfluxDB

In [9]:
from influxdb_client.client.query_api import QueryApi

query_api: QueryApi = client.query_api()

query_flux = f"""
from( bucket: "iot_data" )
 |> range( start: -10m, stop: now() )
 |> filter( fn: (r) => r._field == "cpu_percent" )
"""

results = query_api.query(org="docs", query=query_flux)

for table in results:
    for record in table.records:
        print(record.values)

{'result': '_result', 'table': 0, '_start': datetime.datetime(2025, 12, 16, 9, 2, 58, 691462, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2025, 12, 16, 9, 12, 58, 691462, tzinfo=datetime.timezone.utc), '_time': datetime.datetime(2025, 12, 16, 9, 12, 19, 544113, tzinfo=datetime.timezone.utc), '_value': 0.1, '_field': 'cpu_percent', '_measurement': 'rednimiento_servidor', 'entorno': 'producción', 'host_id': 'servidor-01'}
{'result': '_result', 'table': 0, '_start': datetime.datetime(2025, 12, 16, 9, 2, 58, 691462, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2025, 12, 16, 9, 12, 58, 691462, tzinfo=datetime.timezone.utc), '_time': datetime.datetime(2025, 12, 16, 9, 12, 21, 543467, tzinfo=datetime.timezone.utc), '_value': 0.3, '_field': 'cpu_percent', '_measurement': 'rednimiento_servidor', 'entorno': 'producción', 'host_id': 'servidor-01'}
{'result': '_result', 'table': 0, '_start': datetime.datetime(2025, 12, 16, 9, 2, 58, 691462, tzinfo=datetime.timezone.utc),